In [1]:
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
import gym
env = gym.make('Go9x9-v0')

[2017-10-04 22:16:11,409] Making new env: Go9x9-v0


In [3]:
import importlib

In [4]:
def opponent(position1st, position2nd):
    """Return the opponent move with the position they saw
    
    Go9x9 gym environment makes whites moves automatically, so 
    it is not as easy to capture their state and move for training
    as it is for the black moves.
    This method calculates the position the white player saw and the 
    move they took in that position.
    
    :param positions___: np.array    3x9x9 arrays
    :return: (np.array, int)         (3x9x9 observation array, move integer)
    """
    mask = (position1st - position2nd)[1,:,:]
    if np.all(mask == 0):
        opp_move = 81
    else:
        opp_move = np.argmin(mask.reshape(81))
    
    mask = np.concatenate([np.zeros((1,9,9)), mask.reshape(1,9,9), -mask.reshape(1,9,9)],)
    opp_position = position2nd + mask

    return opp_position, opp_move

In [5]:
opponent(*observations[-2:])

NameError: name 'observations' is not defined

In [29]:
import paths
from nn import policy9x9
importlib.reload(policy9x9)

<module 'nn.policy9x9' from '/Users/andrew.brown/projects/GoFamiliar/src/nn/policy9x9.py'>

In [30]:
net = policy9x9.PolicyNet()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_17 (InputLayer)            (None, 2, 9, 9)       0                                            
____________________________________________________________________________________________________
zero_padding2d_17 (ZeroPadding2D (None, 4, 11, 9)      0           input_17[0][0]                   
____________________________________________________________________________________________________
conv2d_17 (Conv2D)               (None, 2, 9, 32)      2624        zero_padding2d_17[0][0]          
____________________________________________________________________________________________________
zero_padding2d_18 (ZeroPadding2D (None, 4, 11, 32)     0           conv2d_17[0][0]                  
___________________________________________________________________________________________

In [ ]:
moves_made_per_game = []
total_reward = 0

for _ in range(100):
    observations = []
    moves = []
    
    opp_observations = []
    opp_moves = []
    
    obser = env.reset()    
    done = False
    
    while not done:
        observations.append(obser)
        move = net.move(position=obser)
        obser, reward, done, info = env.step(move)
        
        moves.append(move)
        
        opp_obser, opp_move = opponent(observations[-1], obser)
        if opp_move < 81:
            opp_observations.append(opp_obser)
            opp_moves.append(opp_move)
    moves_made_per_game.append(len(moves))
    total_reward += reward
    
    played_on_stone = np.any(observations[-1][2,:,:].reshape((81,)) == 0)
    if played_on_stone:
        obs_before_playing_on_stone = np.array(observations[-1]).reshape(policy9x9.BOARD_SHAPE_1)
        illegal_move = keras.utils.to_categorical(moves[-1], num_classes=len(policy9x9.ACTION_SPACE))
        illegal_playreward = illegal_move * reward
        
        net.train(observations=obs_before_playing_on_stone, 
                  actionrewards=illegal_playreward, 
                  verbose=0)
        reward = -reward
        
    moves = keras.utils.to_categorical(np.array(moves), num_classes=len(policy9x9.ACTION_SPACE))
    actionrewards = moves * reward    # uses the final reward as the reward for all moves
    observations = np.array(observations)
    
    net.train(observations=observations, 
              actionrewards=actionrewards, 
              verbose=0)
    
    opp_moves = keras.utils.to_categorical(np.array(opp_moves), num_classes=len(policy9x9.ACTION_SPACE))
    opp_actionrewards = opp_moves * (-reward)
    opp_observations = np.array(opp_observations)
    
    net.train(observations=opp_observations, 
              actionrewards=opp_actionrewards, 
              verbose=0)

np.median(moves_made_per_game), total_reward

/Users/andrew.brown/projects/GoFamiliar/src/nn/policy9x9.py:120: RuntimeWarning: invalid value encountered in true_divide
  return np.random.choice(ACTION_SPACE, p=probs)
/Users/andrew.brown/projects/GoFamiliar/src/nn/policy9x9.py:121: RuntimeWarning: invalid value encountered in less
  


## Original

In [10]:
np.median(moves_made_per_game), total_reward

(4.0, -89.0)

In [12]:
np.median(moves_made_per_game), total_reward

(4.0, -91.0)

In [14]:
np.median(moves_made_per_game), total_reward

(4.0, -93.0)

## Removed dropout

In [8]:
np.median(moves_made_per_game), total_reward

(6.0, -84.0)

In [10]:
np.median(moves_made_per_game), total_reward

(4.0, -55.0)

In [12]:
np.median(moves_made_per_game), total_reward

(3.0, -70.0)

## Added connection from inputs to end
also reduced the action sapce, so passing was no longer allowed.

In [11]:
np.median(moves_made_per_game), total_reward

(8.5, -100.0)

In [13]:
np.median(moves_made_per_game), total_reward

(6.5, -100.0)

## Split stones from Open board part of position

In [32]:
np.median(moves_made_per_game), total_reward

(20.5, -100.0)

In [35]:
observations[-1]

array([[[0, 1, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 1, 1, 1, 1]],

       [[0, 0, 0, 1, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 1, 0, 1, 0, 0, 0, 0],
        [0, 1, 0, 1, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[1, 0, 1, 0, 1, 0, 0, 1, 1],
        [1, 1, 1, 0, 0, 1, 1, 1, 1],
        [1, 0, 1, 1, 1, 0, 0, 1, 1],
        [0, 1, 0, 0, 0, 1, 1, 1, 1],
        [1, 0, 1, 0, 0, 1, 0, 1, 1],
        [0, 1, 0, 1, 0, 0, 1, 1, 1],
        [1, 0, 1, 0, 1, 1, 1, 1, 0],
        [0, 0, 0, 1, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1, 0, 0, 0, 0

In [37]:
moves[-1].reshape(9,9)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])